## Lista 7 - Exercício de Mineração de texto

#### Questão 1 

Use o dataset "tweets_trump.csv", que contém todos os tweets do D. Trump e faça o que é pedido abaixo.

a) Utilizando o texto dos tweets identifique 5 tópicos latentes com o LDA, com um preprocessamento básico -- remoção de urls, remoção de pontuação, stemming e conversão para minúsculo. 

b) Avalie os tópicos encontrados. Algum preprocessamento a mais poderia ser útil para a produção de um melhor resultado?


## Questão 2

Use o dataset "tweets_trump.csv" e responda:

a) Qual o sentimento dos 5 tweets com maior número de retweets?

b) Faça uma análise do sentimento geral de todos os tweets. Para isso, visualize a distribuição de polaridade para embasar a sua resposta.